Importing Libraries


In [1]:
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn as nn

Loading Dataset


In [3]:
import requests
import zipfile
import os

# URL of the dataset
url = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/concrete_crack_images_for_classification.zip"

# Path to download the file
zip_file_path = "/resources/data/concrete_crack_images_for_classification.zip"

# Ensure the directory exists
os.makedirs(os.path.dirname(zip_file_path), exist_ok=True)

# Download the file
response = requests.get(url)
with open(zip_file_path, "wb") as f:
    f.write(response.content)

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall("/resources/data")

# Optionally, remove the zip file after extraction
os.remove(zip_file_path)


Dataset Class



In [4]:
class Dataset(Dataset):

    # Constructor
    def __init__(self,transform=None,train=True):
        directory="/resources/data"
        positive="Positive"
        negative="Negative"

        positive_file_path=os.path.join(directory,positive)
        negative_file_path=os.path.join(directory,negative)
        positive_files=[os.path.join(positive_file_path,file) for file in  os.listdir(positive_file_path) if file.endswith(".jpg")]
        positive_files.sort()
        negative_files=[os.path.join(negative_file_path,file) for file in  os.listdir(negative_file_path) if file.endswith(".jpg")]
        negative_files.sort()
        number_of_samples=len(positive_files)+len(negative_files)
        self.all_files=[None]*number_of_samples
        self.all_files[::2]=positive_files
        self.all_files[1::2]=negative_files
        # The transform is goint to be used on image
        self.transform = transform
        #torch.LongTensor
        self.Y=torch.zeros([number_of_samples]).type(torch.LongTensor)
        self.Y[::2]=1
        self.Y[1::2]=0

        if train:
            self.all_files=self.all_files[0:10000] #Change to 30000 to use the full test dataset
            self.Y=self.Y[0:10000] #Change to 30000 to use the full test dataset
            self.len=len(self.all_files)
        else:
            self.all_files=self.all_files[30000:]
            self.Y=self.Y[30000:]
            self.len=len(self.all_files)

    # Get the length
    def __len__(self):
        return self.len

    # Getter
    def __getitem__(self, idx):


        image=Image.open(self.all_files[idx])
        y=self.Y[idx]


        # If there is any transform method, apply it onto the image
        if self.transform:
            image = self.transform(image)

        return image, y

Transform Object


In [5]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]


transform =transforms.Compose([ transforms.ToTensor(), transforms.Normalize(mean, std)])


Splitting Dataset

In [6]:
dataset_train=Dataset(transform=transform,train=True)
dataset_val=Dataset(transform=transform,train=False)

Shape & Size


In [ ]:
dataset_train[0][0].shape
size_of_image=3*227*227
size_of_image

In [ ]:
torch.manual_seed(0)

Custom Module


In [15]:
class Softmax(nn.Module):
    def __init__(self,in_size,out_size):
        super(Softmax,self).__init__()
        self.linear = nn.Linear(in_size,out_size)
    def forward(self,x):
        out = self.linear(x)
        return out

Model Object


In [16]:
out_size = 2
model = Softmax(size_of_image,out_size)

Optimizer, Criterion, Learning rate, Momentum


In [18]:
l_r = 0.1
p = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=l_r, momentum = p)
criterion = nn.CrossEntropyLoss()

Data Loader 


In [20]:

train_loader = DataLoader(dataset=dataset_train, batch_size=100)
validation_loader = DataLoader(dataset=dataset_val, batch_size=5000)


Training the Model

In [ ]:
n_epochs = 5
accuracy_list = []
N_test = len(dataset_val)
max_accuracy = 0.0  # Initialize max accuracy

def train_model(n_epochs, size_of_image):
    global max_accuracy

    for epoch in range(n_epochs):
        for x, y in train_loader:
            optimizer.zero_grad()
         
            z = model(x.view(-1, size_of_image))
            loss = criterion(z, y)
            loss.backward()
            optimizer.step()

        correct = 0
        # Perform a prediction on the validation data
        for x_test, y_test in validation_loader:
          
            z = model(x_test.view(-1, size_of_image))
            _, yhat = torch.max(z.data, 1)
            correct += (yhat == y_test).sum().item()

        accuracy = correct / N_test
        accuracy_list.append(accuracy)

        # Update max accuracy if current epoch's accuracy is higher
        if accuracy > max_accuracy:
            max_accuracy = accuracy

train_model(n_epochs, size_of_image)
print(f"Maximum Accuracy Achieved: {max_accuracy}")
